In [62]:
import csv
import imblearn
import matplotlib.pyplot as plt
import mglearn
import numpy as np
from numpy.random import seed
seed(1)

import os
import pandas as pd
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from collections import Counter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix,classification_report, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import joblib

import warnings

In [63]:
df = pd.read_csv("train_preprocessed.csv")
df_test = pd.read_csv("test_preprocessed.csv")

In [64]:
class2idx = {
    1:0,
    2:1,
    3:2,
    4:3,
    5:4,
    6:5,
    7:6,
    8:7
}

idx2class = {v: k for k, v in class2idx.items()}

df['next_month_plan'].replace(class2idx, inplace=True)

In [65]:
ids = df_test['primary_identifier']
df_test = df_test.drop(['Unnamed: 0','primary_identifier'],axis=1)
df_test.columns

Index(['gender', 'network_stay', 'average_monthly_bill_amount', 'dusage_sum',
       'dusage_min', 'dusage_max', 'dusage_avg', 'dusage_days',
       'dusage_stddev', 'vusage_onnet_sum', 'vusage_onnet_max',
       'vusage_onnet_min', 'vusage_onnet_days', 'vusage_onnet_avg',
       'vusage_onnet_stddev', 'vusage_offnet_sum', 'vusage_offnet_max',
       'vusage_offnet_min', 'vusage_offnet_days', 'vusage_offnet_avg',
       'vusage_offnet_stddev', 'number_of_fixed_bb_accounts',
       'number_of_iptv_accounts', 'add_on_tot_rental', 'add_on_count',
       'device_type_2G', 'device_type_3G', 'device_type_4G',
       'device_category_Basic', 'device_category_Feature phone',
       'device_category_M2M', 'device_category_Modem',
       'device_category_Pluggable card', 'device_category_Smartphone',
       'device_category_Tablet', 'district_name_Ampara',
       'district_name_Anuradhapura', 'district_name_Badulla',
       'district_name_Batticaloa', 'district_name_Colombo',
       'district_na

In [66]:
y = df.next_month_plan
df = df.drop(['Unnamed: 0','next_month_plan','primary_identifier'],axis=1)
df.columns

Index(['gender', 'network_stay', 'average_monthly_bill_amount', 'dusage_sum',
       'dusage_min', 'dusage_max', 'dusage_avg', 'dusage_days',
       'dusage_stddev', 'vusage_onnet_sum', 'vusage_onnet_max',
       'vusage_onnet_min', 'vusage_onnet_days', 'vusage_onnet_avg',
       'vusage_onnet_stddev', 'vusage_offnet_sum', 'vusage_offnet_max',
       'vusage_offnet_min', 'vusage_offnet_days', 'vusage_offnet_avg',
       'vusage_offnet_stddev', 'number_of_fixed_bb_accounts',
       'number_of_iptv_accounts', 'add_on_tot_rental', 'add_on_count',
       'device_type_2G', 'device_type_3G', 'device_type_4G',
       'device_category_Basic', 'device_category_Feature phone',
       'device_category_Modem', 'device_category_Pluggable card',
       'device_category_Smartphone', 'device_category_Tablet',
       'district_name_Ampara', 'district_name_Anuradhapura',
       'district_name_Badulla', 'district_name_Batticaloa',
       'district_name_Colombo', 'district_name_Galle', 'district_name_Gamp

In [67]:
X = df.iloc[:,:]

# ids = df_test.primary_identifier
X.shape

(10500, 68)

In [68]:
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
rus = RandomUnderSampler(sampling_strategy={0:1200, 2:900, 4:780, 1:677, 5:592, 3:588, 6: 362, 7: 351})
#rus.fit(X, y)
X_resampled, y_resampled = rus.fit_resample(X, y)
# X = X_resampled
# y = y_resampled
print('Resampled dataset shape %s' % Counter(y_resampled))
print('Before Resampled dataset shape %s' % Counter(y))

Resampled dataset shape Counter({0: 1200, 2: 900, 4: 780, 1: 677, 5: 592, 3: 588, 6: 362, 7: 351})
Before Resampled dataset shape Counter({0: 3673, 2: 1717, 4: 1558, 1: 1354, 5: 740, 3: 735, 6: 364, 7: 359})


In [69]:

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, stratify=y_resampled, random_state=len(X_resampled))

# Split train into train-val
# X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.1, stratify=y_trainval, random_state=21)

In [70]:
df.columns

Index(['gender', 'network_stay', 'average_monthly_bill_amount', 'dusage_sum',
       'dusage_min', 'dusage_max', 'dusage_avg', 'dusage_days',
       'dusage_stddev', 'vusage_onnet_sum', 'vusage_onnet_max',
       'vusage_onnet_min', 'vusage_onnet_days', 'vusage_onnet_avg',
       'vusage_onnet_stddev', 'vusage_offnet_sum', 'vusage_offnet_max',
       'vusage_offnet_min', 'vusage_offnet_days', 'vusage_offnet_avg',
       'vusage_offnet_stddev', 'number_of_fixed_bb_accounts',
       'number_of_iptv_accounts', 'add_on_tot_rental', 'add_on_count',
       'device_type_2G', 'device_type_3G', 'device_type_4G',
       'device_category_Basic', 'device_category_Feature phone',
       'device_category_Modem', 'device_category_Pluggable card',
       'device_category_Smartphone', 'device_category_Tablet',
       'district_name_Ampara', 'district_name_Anuradhapura',
       'district_name_Badulla', 'district_name_Batticaloa',
       'district_name_Colombo', 'district_name_Galle', 'district_name_Gamp

In [71]:
cat_columns = []
num_columns = ['network_stay', 'average_monthly_bill_amount',
 'dusage_sum', 'dusage_min', 'dusage_max', 'dusage_avg',
  'dusage_days', 'dusage_stddev', 'vusage_onnet_sum',
   'vusage_onnet_max', 'vusage_onnet_min', 'vusage_onnet_days',
    'vusage_onnet_avg', 'vusage_onnet_stddev', 'vusage_offnet_sum',
     'vusage_offnet_max', 'vusage_offnet_min', 'vusage_offnet_days',
      'vusage_offnet_avg', 'vusage_offnet_stddev', 'number_of_fixed_bb_accounts',
       'number_of_iptv_accounts', 'add_on_tot_rental', 'add_on_count','gender','device_type_2G', 'device_type_3G', 'device_type_4G',
       'device_category_Basic', 'device_category_Feature phone',
       'device_category_Modem', 'device_category_Pluggable card',
       'device_category_Smartphone', 'device_category_Tablet',
       'district_name_Ampara', 'district_name_Anuradhapura',
       'district_name_Badulla', 'district_name_Batticaloa',
       'district_name_Colombo', 'district_name_Galle', 'district_name_Gampaha',
       'district_name_Hambantota', 'district_name_Jaffna',
       'district_name_Kalutara', 'district_name_Kandy',
       'district_name_Kegalle', 'district_name_Kilinochchi',
       'district_name_Kurunegala', 'district_name_Mannar',
       'district_name_Matale', 'district_name_Matara',
       'district_name_Monaragala', 'district_name_Mullaitivu',
       'district_name_Nuwara Eliya', 'district_name_Polonnaruwa',
       'district_name_Puttalam', 'district_name_Ratnapura',
       'district_name_Trincomalee', 'district_name_Unknown',
       'district_name_Vavuniya', 'age_group_20-30', 'age_group_30-40',
       'age_group_40-50', 'age_group_50-60', 'age_group_60-70',
       'age_group_<20', 'age_group_>70', 'device_category_M2M']

In [72]:
categorical_pipeline = Pipeline([
    ('encoder', OneHotEncoder())
])

numerical_pipeline = Pipeline([
    ('scaler', RobustScaler())
])

prepocessor = ColumnTransformer([
    ('categorical',categorical_pipeline,cat_columns),
    ('numerical', numerical_pipeline,num_columns)
])

pipe_logreg = Pipeline([
    ("prep", prepocessor),
    ("algo", LogisticRegression())
])

pipe_KNN = Pipeline([
    ('prep', prepocessor),
    ('algo', KNeighborsClassifier(n_neighbors=10))
])


pipe_RF = Pipeline([
    ("prep", prepocessor),
    ("algo", RandomForestClassifier())
])

pipe_DT = Pipeline([
    ("prep", prepocessor),
    ("algo", DecisionTreeClassifier())
])

pipe_XGB = Pipeline([
    ("prep", prepocessor),
    ("algo", XGBClassifier())
])


In [73]:
from sklearn import svm

clf = svm.LinearSVC(penalty='l2', loss='squared_hinge', dual=True,
 tol=0.001, C=2.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1,
  class_weight=None, verbose=0, random_state=23, max_iter=10000)
clf.fit(X_train, y_train)  
prclf = clf.predict(X_test)

In [74]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,prclf))
print(classification_report(y_test,prclf))

[[302   8  27   4  15   1   2   1]
 [144   2  38   3  12   0   1   3]
 [112   4  99  13  30   6   1   5]
 [ 54   6  69  13  22   7   0   5]
 [ 28   1  10   2 156  26   6   5]
 [ 15   4  11   1  90  35   6  16]
 [  7   2   6   0  38  27   5  24]
 [  5   2   5   4  29  29   7  24]]
              precision    recall  f1-score   support

           0       0.45      0.84      0.59       360
           1       0.07      0.01      0.02       203
           2       0.37      0.37      0.37       270
           3       0.33      0.07      0.12       176
           4       0.40      0.67      0.50       234
           5       0.27      0.20      0.23       178
           6       0.18      0.05      0.07       109
           7       0.29      0.23      0.26       105

    accuracy                           0.39      1635
   macro avg       0.29      0.30      0.27      1635
weighted avg       0.32      0.39      0.32      1635



In [75]:
pipe_KNN.fit(X_train, y_train)
y_pred_KNN_base =  pipe_KNN.predict(X_test)
y_pred_KNN_base_train = pipe_KNN.predict(X_train)

list(y_pred_KNN_base_train)

# 

[5,
 5,
 4,
 4,
 0,
 6,
 6,
 4,
 0,
 5,
 0,
 2,
 5,
 1,
 0,
 2,
 3,
 5,
 4,
 0,
 0,
 5,
 4,
 0,
 7,
 1,
 3,
 0,
 3,
 2,
 4,
 4,
 4,
 2,
 4,
 2,
 1,
 1,
 0,
 5,
 0,
 4,
 5,
 4,
 3,
 1,
 5,
 5,
 3,
 4,
 6,
 4,
 4,
 5,
 0,
 0,
 0,
 4,
 7,
 5,
 2,
 3,
 5,
 5,
 0,
 2,
 0,
 1,
 0,
 7,
 3,
 2,
 1,
 4,
 2,
 2,
 0,
 4,
 0,
 0,
 1,
 4,
 0,
 4,
 3,
 2,
 0,
 0,
 4,
 4,
 0,
 5,
 3,
 3,
 2,
 5,
 0,
 5,
 4,
 5,
 0,
 1,
 5,
 2,
 4,
 3,
 4,
 0,
 0,
 0,
 2,
 2,
 6,
 4,
 0,
 6,
 4,
 1,
 0,
 7,
 5,
 5,
 5,
 3,
 4,
 5,
 4,
 2,
 1,
 5,
 0,
 0,
 2,
 0,
 0,
 0,
 4,
 6,
 1,
 4,
 5,
 6,
 4,
 3,
 2,
 2,
 5,
 0,
 6,
 5,
 4,
 0,
 5,
 0,
 2,
 5,
 0,
 2,
 6,
 2,
 1,
 0,
 4,
 1,
 0,
 0,
 4,
 5,
 0,
 0,
 2,
 7,
 2,
 4,
 2,
 1,
 5,
 6,
 2,
 3,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 4,
 0,
 6,
 4,
 0,
 4,
 0,
 1,
 0,
 0,
 0,
 4,
 4,
 4,
 2,
 6,
 3,
 3,
 1,
 6,
 1,
 0,
 4,
 4,
 3,
 6,
 4,
 4,
 4,
 4,
 0,
 1,
 1,
 1,
 1,
 0,
 4,
 4,
 2,
 3,
 0,
 4,
 0,
 6,
 5,
 4,
 0,
 2,
 4,
 0,
 0,
 4,
 2,
 0,
 4,
 7,
 0,
 4,
 0,
 4,
 0,
 4,


In [76]:
recall_KNN_base = recall_score(y_test, y_pred_KNN_base, average= 'macro')
acc_KNN_base = accuracy_score(y_test, y_pred_KNN_base)
precision_KNN_base = precision_score(y_test, y_pred_KNN_base,average= 'macro')
f1_KNN_base = f1_score(y_test, y_pred_KNN_base,average= 'macro')
acc_KNN_base_train = accuracy_score(y_train, y_pred_KNN_base_train)

print(f"Training Accuracy : {acc_KNN_base_train}")
print(f"Testing Accuracy : {acc_KNN_base}")
print(f"F1 Score : {f1_KNN_base}")

# in KNN model we see that there is overfitting condition in KNN model
# even though KNN has a better accuracy score compared to logistic Regression
# we will fix it later on in hyperparameter tuning

Training Accuracy : 0.4705111402359109
Testing Accuracy : 0.3296636085626911
F1 Score : 0.2826450131382038


In [77]:
pipe_DT.fit(X_train, y_train)
y_pred_DT_base =  pipe_DT.predict(X_test)
y_pred_DT_base_train = pipe_DT.predict(X_train)

In [78]:
recall_DT_base = recall_score(y_test, y_pred_DT_base)
acc_DT_base = accuracy_score(y_test, y_pred_DT_base)
precision_DT_base = precision_score(y_test, y_pred_DT_base)
f1_DT_base = f1_score(y_test, y_pred_DT_base)
acc_DT_base_train = accuracy_score(y_train, y_pred_DT_base_train)

print(f"Training Accuracy : {acc_DT_base_train}")
print(f"Testing Accuracy : {acc_DT_base}")

# we see that there's overfitting condition in Decision Tree as well 
# however we will try to adjust the parameter to prevent the overfitting condition

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
gnb = GaussianNB()
gnb

GaussianNB()

In [ ]:
y_pred = gnb.fit(X_train, y_train).predict(X_test)
y_pred

array([7, 6, 7, ..., 7, 7, 7], dtype=int64)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[  5  21   3  41   0   0  64 226]
 [  1   8   5  25   1   0  23 140]
 [  0   6   6  48   4   0  30 176]
 [  0   1   3  31   0   0  21 120]
 [  1   1   3   9   3   0  31 186]
 [  0   2   0   5   1   0  28 142]
 [  1   1   1   6   2   0  11  87]
 [  0   2   0   3   1   0  16  83]]
              precision    recall  f1-score   support

           0       0.62      0.01      0.03       360
           1       0.19      0.04      0.07       203
           2       0.29      0.02      0.04       270
           3       0.18      0.18      0.18       176
           4       0.25      0.01      0.02       234
           5       0.00      0.00      0.00       178
           6       0.05      0.10      0.07       109
           7       0.07      0.79      0.13       105

    accuracy                           0.09      1635
   macro avg       0.21      0.14      0.07      1635
weighted avg       0.27      0.09      0.06      1635



C:\Users\dewth\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dewth\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dewth\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
